In [1]:
import os

In [2]:
%pwd

'H:\\PU\\PRJ\\kidney-disease-classification\\kidney-disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'H:\\PU\\PRJ\\kidney-disease-classification\\kidney-disease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_files : Path
    unzip_dir : Path

In [7]:
from classifier.constants import *
from classifier.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_files = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [9]:
import os
import zipfile
import gdown
from classifier import logger
from classifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        """
        Fetch data from the Source URL
        :return: 
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_files
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")
            
            file_name = dataset_url.split("/")[-2]
            prefix_url = "https://drive.google.com/uc?/export=download&id={}".format(file_name)
            gdown.download(prefix_url, zip_download_dir)
            logger.info(f"Extracting data from {dataset_url} to {zip_download_dir}")
        except Exception as e:
            logger.error(f"{e}")
            raise e
    
    def extract_zip_files(self):
        """
        Extract the zip files in the artifacts directory
        :param: zip_file_path: path of the zip file
        :return: None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_files, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"File unzipped at {unzip_path}")

In [15]:
try:
    config = ConfigurationManager()
    logger.info(f"Configuration loading completed")
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_files()
except Exception as e:
    raise e

[2024-04-09 09:47:47,547 : INFO : common : Loaded config from config\config.yaml successfully]
[2024-04-09 09:47:47,549 : INFO : common : Loaded config from params.yaml successfully]
[2024-04-09 09:47:47,550 : INFO : common : Created directory at artifacts]
[2024-04-09 09:47:47,551 : INFO : 3635039260 : Configuration loading completed]
[2024-04-09 09:47:47,552 : INFO : common : Created directory at artifacts/data_ingestion]
[2024-04-09 09:47:47,553 : INFO : 2318707573 : Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=07f9d72b-9b35-4ed7-81df-326fa4f25c52
To: H:\PU\PRJ\kidney-disease-classification\kidney-disease-classification\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:18<00:00, 3.07MB/s]

[2024-04-09 09:48:07,818 : INFO : 2318707573 : Extracting data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing to artifacts/data_ingestion/data.zip]


[2024-04-09 09:48:08,415 : INFO : 2318707573 : File unzipped at artifacts/data_ingestion]
